In [18]:
import re
import asyncio
import yaml
import time

import itertools
from typing import List, Set, Dict


import aiohttp
from aiohttp import ClientSession

from time import sleep
import selenium
import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from adbot.conf.settings import Settings
from adbot.misc.website import Website, URL
from adbot.misc.files import Video
from adbot.core.download import download_video

from adbot import HEADERS

In [2]:
site = URL("www.tv2.dk")
site2 = URL("tv2.dk")
site3 = URL("www.tv2.dk")

In [3]:
settings = Settings.load("tv2")

In [4]:
settings.tags

['iframe', 'video', 'lima-video']

In [4]:
l = [site, site2, site3]

In [5]:
set(l)

{URL<https://tv2.dk>, URL<https://www.tv2.dk>}

In [6]:
site + "/new"

In [7]:
site

URL<https://www.tv2.dk/new>

In [8]:
resp = requests.get(site)

In [9]:
website = Website(site)

## YAML

In [10]:
with open("adbot/settings/default.yml", mode="r", encoding="utf-8") as file:
    print(yaml.safe_load(file))

{'version': 0.1, 'environment': 'development', 'level': 2, 'files': [{'image': ['.png', '.jpg', '.jpeg', '.svg']}, {'video': ['.mp3', '.mp4']}], 'tags': ['iframe']}


In [10]:
file = yaml.SafeLoader("adbot/settings/default.yml")

In [11]:
print(file)

## Requests

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [6]:
url = "https://tv2.dk/"

In [7]:
response = requests.get(url)

In [8]:
html = BeautifulSoup(response.text)

In [9]:
def validate_link(link: str, domain: str) -> str:
    length = len("https://")
    if bool(re.search(domain, link)):
        if link[:length] == "https":
            return link
        else:
            return "https://" + domain + link
    return None

In [11]:
link = 'https://tv2.dk/navigation'
#link = "main"

In [12]:
match = bool(re.search("tv2.dk", link))

In [13]:
match

True

In [14]:
urls = set(i.get("href") for i in html.find_all("a") if validate_link(i.get("href"), "tv2.dk") is not None)

In [16]:
urls = list(urls)
len(urls)

102

In [17]:
%%time

async def fetch_url(url: URL, session: ClientSession) -> str:
    print(f"[Info] Fetch {str(url)}")
    response = await session.request(method="GET", url=url)
    print(f"[Info] {str(url)} responded with status code {response.status}")
    html = await response.text()
    return html


async def collect_links(html: str) -> List[str]:
    href = re.compile(r'href="(.*?)"')
    _links = list(link for link in href.findall(html))
    return _links


async def fetch_links(urls: List[URL]) -> List[str]:
    """
    Usage:
        output = asyncio.run(fetch_links(urls=urls))
    """
    links = list()

    async def update_links(url: URL, session: ClientSession) -> None:
        html = await fetch_url(url=url, session=session)
        _links = await collect_links(html=html)
        print(f"[Info] On {str(url)} {len(_links)} links were found")
        return _links

    async with ClientSession() as session:
        tasks = list(update_links(url=url, session=session) for url in urls)
        _links = await asyncio.gather(*tasks)
        links += _links
    return links

output = asyncio.run(fetch_links(urls=urls))

[Info] Fetch https://tv.tv2.dk/2017-08-12-tv-2-apps-tv-2-play-nyheder-sport-vejr-og-tv
[Info] Fetch https://tv2.dk
[Info] Fetch https://mit.tv2.dk
[Info] Fetch https://nyheder.tv2.dk/udland/2022-05-28-danske-missiler-kan-beskytte-vigtig-kystby-vurderer-militaeranalytiker
[Info] Fetch https://nyheder.tv2.dk/udland/2022-05-28-vigtig-allieret-vender-putin-ryggen-klart-budskab-til-moskva-siger-forsker
[Info] Fetch https://nyheder.tv2.dk/udland/2022-05-28-mirror-boris-johnson-vil-genindfore-gammelt-malesystem
[Info] Fetch https://sport.tv2.dk/haandbold/2022-05-28-esbjerg-udskyder-dm-afgoerelse-med-overlegen-sejr
[Info] Fetch https://sport.tv2.dk/haandbold/2022-05-28-profil-i-taarer-efter-sidste-hjemmekamp
[Info] Fetch https://nyheder.tv2.dk/udland/2022-05-29-joe-biden-mindes-ofrene-for-skoleskydningen-i-uvalde
[Info] Fetch https://sport.tv2.dk/formel-1/2022-05-28-magnussen-skuffer-i-kvalifikationen
[Info] Fetch https://sport.tv2.dk/
[Info] Fetch https://sport.tv2.dk/formel-1/2022-05-28-magn

[Info] https://tv.tv2.dk/2017-08-12-tv-2-apps-tv-2-play-nyheder-sport-vejr-og-tv responded with status code 200
[Info] https://tv.tv2.dk/ responded with status code 200
[Info] https://tv2.dk responded with status code 200
[Info] https://nyheder.tv2.dk/udland/2022-05-28-danske-missiler-kan-beskytte-vigtig-kystby-vurderer-militaeranalytiker responded with status code 200
[Info] https://nyheder.tv2.dk/udland/2022-05-28-mirror-boris-johnson-vil-genindfore-gammelt-malesystem responded with status code 200
[Info] https://nyheder.tv2.dk/udland/2022-05-29-son-af-britisk-politiker-slas-i-ukraine responded with status code 200
[Info] https://nyheder.tv2.dk/politik/2022-05-29-ny-maaling-tegner-klart-billede-faa-dage-foer-folkeafstemningen responded with status code 200
[Info] https://nyheder.tv2.dk/politik/2022-05-11-alt-du-skal-vide-om-forsvarsforbeholdet responded with status code 200
[Info] https://nyheder.tv2.dk/udland/2022-05-28-putin-vi-er-klar-til-at-drofte-korneksport-fra-ukraine responde

[Info] On https://sport.tv2.dk/haandbold/2022-05-28-profil-i-taarer-efter-sidste-hjemmekamp 90 links were found
[Info] On https://kundeservice.tv2.dk/ 15 links were found
[Info] https://omtv2.tv2.dk/kontakt/eksterne-partnere/offentlig-fremvisning responded with status code 200
[Info] On https://vejr.tv2.dk/2022-05-29-se-op-mulighed-for-skypumper-paa-himlen 78 links were found
[Info] On https://play.tv2.dk 46 links were found
[Info] On https://nyheder.tv2.dk/politik/2022-05-21-hvad-mener-partierne-om-forsvarsforbeholdet-faa-overblikket-her 79 links were found
[Info] On https://sport.tv2.dk/formel-1/2022-05-29-magnussen-forstaar-intet-af-afgoerende-beslutning 72 links were found
[Info] On https://sport.tv2.dk/resultater 117 links were found
[Info] On https://nyheder.tv2.dk/udland/2022-05-29-zelenskyj-forlader-for-foerste-gang-siden-krigens-start-kyiv-omraadet-staerk 87 links were found
[Info] On https://sport.tv2.dk/tennis/2022-05-28-rune-udraderer-modstander-ved-french-open 86 links wer

[Info] On https://nyheder.tv2.dk/udland/2022-05-29-russisk-ambassador-udelukker-brug-af-atomvaben-i-ukraine 84 links were found
[Info] On https://nyheder.tv2.dk/udland/2022-05-28-putin-vi-er-klar-til-at-drofte-korneksport-fra-ukraine 83 links were found
[Info] On https://nyheder.tv2.dk/udland/2022-05-28-danske-missiler-kan-beskytte-vigtig-kystby-vurderer-militaeranalytiker 96 links were found
[Info] On https://nyheder.tv2.dk/krimi/2022-05-29-kajakroere-finder-livlos-mand-i-vandet-ud-for-kastrup 83 links were found
[Info] On https://nyheder.tv2.dk/politik/2022-05-28-ja-partierne-kan-se-danmark-deltage-i-flere-eu-missioner-men-har-forsvaret 84 links were found
[Info] https://sport.tv2.dk/formel-1/2022-05-28-magnussen-skuffer-i-kvalifikationen responded with status code 200
[Info] On https://sport.tv2.dk/formel-1/2022-05-28-magnussen-skuffer-i-kvalifikationen 76 links were found
[Info] On https://omtv2.tv2.dk/ 88 links were found
[Info] https://play.tv2.dk/programmer/magasiner/serier/kern

In [36]:
merged = list(itertools.chain(*output))

In [38]:
len(merged)

9029

## Find by iframes

In [19]:
options = webdriver.FirefoxOptions()
#options.add_argument('--headless')
#options.add_argument('--no-sandbox')

In [20]:
fox = webdriver.Firefox(options=options)
fox.get(site.url)
WebDriverWait(fox, 10).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div/div/div/div[2]/div/button[3]"))
    ).click()
#WebDriverWait(fox, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "iframe")))
#fox.find_element(by=By.TAG_NAME, value="html").send_keys(Keys.END)
iframes = fox.find_elements_by_tag_name('iframe')
len(iframes)

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_13686/41994048.py:8: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  iframes = fox.find_elements_by_tag_name('iframe')


1

In [40]:
html = BeautifulSoup(fox.page_source)

In [37]:
fox.find_element_by_tag_name('iframe').

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_13686/2017387176.py:1: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  fox.find_element_by_tag_name('iframe').id


'e6bc1e46-00ed-42e1-ac6e-82b94b53c41b'

In [23]:
iframe = iframes[0]

In [38]:
iframe.id

'5c7e24f8-73fe-4ef8-8d21-e0b0b8822d84'

In [62]:
iframes = fox.find_elements_by_tag_name('iframe')
len(iframes)

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_12291/34575999.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  iframes = fox.find_elements_by_tag_name('iframe')


13

In [44]:
len(html.find_all("iframe"))

9

In [51]:
iframes = fox.find_elements_by_tag_name('iframe')

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_13686/3196645008.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  iframes = fox.find_elements_by_tag_name('iframe')


In [59]:
iframes[4].id

'a0e1741d-1933-4fba-8161-9701eadd2c7c'

In [50]:
list(i.get("id") for i in html.find_all("iframe"))

['google_ads_iframe_/19784602/tv2dk-web/Forsiden/Forsiden_Forside/Forsiden_Forside_Topbanner_1_0',
 'google_ads_iframe_/19784602/tv2dk-web/Forsiden/Forsiden_Forside/Forsiden_Forside_Wide_1_0',
 None,
 None,
 'criteo-syncframe-pbt',
 'gemius_hcconn_1653847395497_63196973',
 None,
 '9144a111ea58852',
 '9276ca58a3b953e8']

In [41]:
SCROLL_PAUSE_TIME = 2 
# Get scroll height
last_height = fox.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    fox.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = fox.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [49]:
SCROLL_HEIGHT = 1000
SCROLL_PAUSE_TIME = 2 

for timer in range(0, 20):
    fox.execute_script("window.scrollTo(0, "+str(SCROLL_HEIGHT)+")")
    SCROLL_HEIGHT += 1000  
    time.sleep(SCROLL_PAUSE_TIME)

In [50]:
iframes = fox.find_elements_by_tag_name('iframe')
len(iframes)

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_12291/34575999.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  iframes = fox.find_elements_by_tag_name('iframe')


13

In [23]:
#fox.find_element_by_css_selector("#qc-cmp2-ui .qc-cmp2-summary-buttons button").click()

In [63]:
fox.quit()

In [60]:
#elems = fox.find_elements(By.TAG_NAME, 'button')

In [14]:
iframes = fox.find_elements_by_tag_name('iframe')

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_9083/3196645008.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  iframes = fox.find_elements_by_tag_name('iframe')


In [16]:
iframe = iframes[0]
fox.switch_to.frame(iframe)

In [41]:
_id.get_dom_attribute

<bound method WebElement.get_dom_attribute of <selenium.webdriver.remote.webelement.WebElement (session="7e4c6342-eb7d-4107-b4f3-3fa49390f383", element="f7070913-de48-4f02-9eb1-b39abe9817e6")>>

In [2]:
video = Video(src="https://s1.adform.net/Banners/Elements/Files/2050865/11254835.mp4?bv=0&amp;ADFAssetID=11254835", name="test.mp4")

In [3]:
download_video(video)

  0%|                                                                                                                                                                          | 0.00/2.20M [00:00<?, ?iB/s]
  7%|████████████▏                                                                                                                                                      | 164k/2.20M [00:00<00:03, 617kiB/s]
 21%|█████████████████████████████████▊                                                                                                                                | 459k/2.20M [00:00<00:01, 1.28MiB/s]
 28%|█████████████████████████████████████████████▌                                                                                                                     | 614k/2.20M [00:00<00:02, 693kiB/s]
 41%|██████████████████████████████████████████████████████████████████▍                                                                                               | 901k/2.20M 

In [57]:
html = BeautifulSoup(fox.page_source)

In [60]:
html.find_all("video")

[<video autoplay="" class="vjs-tech" id="tv2-player-1653819302419_html5_api" muted="muted" playsinline="playsinline" poster="https://cdn-free-vod-new.tv2i.dk/mpx/186373j1jr_640x360_2037552195929.png?w=640&amp;h=360&amp;q=60&amp;auto=format" preload="none" tabindex="-1"></video>,
 <video playsinline="true" src="https://redirector.gvt1.com/videoplayback/id/d3bbdb157a42d2a2/itag/109/source/dclk_video_ads/requiressl/yes/acao/yes/mime/video%2Fmp4/ctier/L/ip/0.0.0.0/ipbits/0/expire/1653840904/sparams/ip,ipbits,expire,id,itag,source,requiressl,acao,mime,ctier/signature/9C3D758BD70C05375E4628C5B53F0E6F849437D4.6335A063FBED8E35ED99FCB078F8766EE5F588E1/key/ck2/file/file.mp4" style="background-color: rgb(0, 0, 0); position: absolute; width: 100%; height: 100%; left: 0px; top: 0px;" title="Advertisement" webkit-playsinline="true"></video>,
 <video playsinline="true" style="background-color: rgb(0, 0, 0); position: absolute; width: 100%; height: 100%; left: 0px; top: 0px;" title="Advertisement" web

In [63]:
html.find_all("video")

[<video autoplay="" class="vjs-tech" id="tv2-player-1653819302419_html5_api" muted="muted" playsinline="playsinline" poster="https://cdn-free-vod-new.tv2i.dk/mpx/186373j1jr_640x360_2037552195929.png?w=640&amp;h=360&amp;q=60&amp;auto=format" preload="none" tabindex="-1"></video>,
 <video playsinline="true" src="https://redirector.gvt1.com/videoplayback/id/d3bbdb157a42d2a2/itag/109/source/dclk_video_ads/requiressl/yes/acao/yes/mime/video%2Fmp4/ctier/L/ip/0.0.0.0/ipbits/0/expire/1653840904/sparams/ip,ipbits,expire,id,itag,source,requiressl,acao,mime,ctier/signature/9C3D758BD70C05375E4628C5B53F0E6F849437D4.6335A063FBED8E35ED99FCB078F8766EE5F588E1/key/ck2/file/file.mp4" style="background-color: rgb(0, 0, 0); position: absolute; width: 100%; height: 100%; left: 0px; top: 0px;" title="Advertisement" webkit-playsinline="true"></video>,
 <video playsinline="true" style="background-color: rgb(0, 0, 0); position: absolute; width: 100%; height: 100%; left: 0px; top: 0px;" title="Advertisement" web

In [37]:
iframes = fox.find_elements_by_tag_name('iframe')

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_12291/3196645008.py:1: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  iframes = fox.find_elements_by_tag_name('iframe')


In [39]:
print(len(iframes))
for idx, iframe in enumerate(iframes):
    try:
        iframe.screenshot(str(idx) + ".jpeg")
    except Exception as e:
        print(e)

1
Message: [Exception... "Failure"  nsresult: "0x80004005 (NS_ERROR_FAILURE)"  location: "JS frame :: chrome://remote/content/marionette/capture.js :: capture.canvas :: line 143"  data: no]
Stacktrace:
capture.canvas@chrome://remote/content/marionette/capture.js:143:62
takeScreenshot@chrome://remote/content/marionette/actors/MarionetteCommandsParent.jsm:256:32



/usr/local/lib/python3.9/site-packages/selenium/webdriver/remote/webelement.py:690: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  warnings.warn("name used for saved screenshot does not match file "


In [25]:
fox.switch_to.frame(fox.find_element_by_tag_name("iframe"))

/var/folders/st/1wj362gj7s7453bxl43k38780000gn/T/ipykernel_2801/2859000034.py:1: DeprecationWarning: find_element_by_tag_name is deprecated. Please use find_element(by=By.TAG_NAME, value=name) instead
  fox.switch_to.frame(fox.find_element_by_tag_name("iframe"))


In [27]:
fox.switch_to.default_content()

### Screenshot

In [38]:
from selenium import webdriver
from PIL import Image
from io import StringIO, BytesIO
import base64

fox = webdriver.Firefox(options=options)
#fox = webdriver.Firefox()
fox.get('http://tv2.dk/')
sleep(3)
fox.find_element(by=By.XPATH, value="/html/body/div[1]/div/div/div/div[2]/div/button[3]").click()

element = fox.find_element(by=By.CLASS_NAME, value='tc_icon__play') # find part of the page you want image of
location = element.location
size = element.size
element.screenshot('screenshot.png')

True

In [40]:
element = fox.find_element(by=By.CLASS_NAME, value='tc_ad')

In [41]:
element.screenshot("screenshot.png")

True

In [114]:
from selenium.webdriver.common.keys import Keys
html = fox.find_element(by=By.TAG_NAME, value="html")
html.send_keys(Keys.END)

## Scroll indefinitely

https://stackoverflow.com/questions/20986631/how-can-i-scroll-a-web-page-using-selenium-webdriver-in-python

For Facebook & LinkedIn stuff

In [ ]:
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height